# 발표 소단원 리스트

9-3, 9-7, 9-11

# 전체 요약자료

- 9-3 : 데코레이터가 적용된 함수가 있는데, 그 함수를 undo 하고 원래 함수를 쓰고 싶을 때
- 9-7 : 함수의 인자를 type checking 하고싶을 때
- 9-11 : wrapped function에 별도의 argument를 추가하고 싶은데, 원래의 calling convention을 방해하고 싶지는 않을 때

### 9-3) Unwrapping a Decorator
- 데코레이터가 적용된 함수가 있는데, 그 함수를 undo 하고 원래 함수를 쓰고 싶을 때

- __ wrapped __ 를 사용하면 원래 함수에 접근 가능합니다

In [45]:
from functools import wraps

def deco(func):
    @wraps(func)
    def wrapper(*args, **kargs):
        print("wraping")
        print(func(*args, **kargs))
        print("======")
        return func(*args, **kargs)
    return wrapper

@deco
def add(arg1, arg2):
    return arg1 + arg2

add(1, 2)

wraping
3


3

In [50]:
orig_add = add.__wrapped__
orig_add(1, 2)

3

- functools에서 반드시 wraps imports 해와서 @wraps annotation 붙여줘야 합니다. 안그러면 @deco annotation 붙인 함수 (위 코드에서는 add)에 __ wrapped__ attribute가 안생깁니다
- 그래서 @wraps가 안붙은 built-in 메서드인 @staticmethod 와 @classmethod같은 것들은 저렇게 못씁니다. 대신 얘내는 __ func __ 에 원래 함수를 저장하도록 합니다

### 9-7) Enforcing Type Checking on a Function Using a Decorator 
- 함수의 인자를 type checking 하고싶을 때

In [ ]:
@typeassert(int, int)
def add(x, y): 
    return x + y

- 위의 방식처럼 decorator를 써서 type체크를 해보고 싶다는 아이디어! 그러면 이제 typeasssert를 구현해봅시다.

In [112]:
from inspect import signature 
from functools import wraps

def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):  
        # If in optimized mode, disable type checking    
        if not __debug__:   
            return func

        # Map function argument names to supplied types    
        sig = signature(func)  
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments
        
        @wraps(func)
        def wrapper(*args, **kwargs):  
            bound_values = sig.bind(*args, **kwargs)
            # Enforce type assertions across supplied arguments    
            for name, value in bound_values.arguments.items():       
                if name in bound_types:   
                    if not isinstance(value, bound_types[name]):    # isinstacne : value가 bound_type인지 알아봅니다
                        raise TypeError(                      
                            'Argument {} must be {}'.format(name, bound_types[name])    
                        )           
            return func(*args, **kwargs)    
        return wrapper    
    return decorate 

In [113]:
@typeassert(int, z=int)
def spam(x, y, z=42): 
    print(x, y, z)
    
spam(1,2,3)
spam(1,"hello",3)
spam(1,2,"hello")

1 2 3
1 hello 3


TypeError: Argument z must be <class 'int'>

In [114]:
sig = signature(spam)
sig

<Signature (x, y, z=42)>

In [115]:
a = 1
b = 2

# Works the same way as Signature.bind(), but allows the omission of some required arguments 
# (mimics functools.partial() behavior.)
print(sig.bind_partial(a, b))
print(sig.bind_partial(a, b).arguments)

<BoundArguments (x=1, y=2)>
OrderedDict([('x', 1), ('y', 2)])


In [117]:
bind_types = sig.bind(a,b)
print(sig.bind(a, b))
print(sig.bind(a, b).arguments.items())

<BoundArguments (x=1, y=2)>
odict_items([('x', 1), ('y', 2)])


In [119]:
isinstance(1, int)

True

### 9-11) Writing Decorators That Add Arguments to Wrapped Functions
- wrapped function에 별도의 argument를 추가하고 싶은데, 원래의 calling convention을 방해하고 싶지는 않을 때
- keyword-only argument의 형태로 주입할 수 있습니다

In [60]:
from functools import wraps

def optional_debug(func):
    @wraps(func) 
    def wrapper(*args, debug=False, **kwargs): 
        if debug:   
            print('Calling', func.__name__)  
        return func(*args, **kwargs) 
    return wrapper

@optional_debug 
def spam(a,b,c): 
    print(a,b,c)

In [61]:
spam(1, 2, 3)
print("==========")
spam(1, 2, 3, debug=True)

1 2 3
Calling spam
1 2 3


- 데코레이터의 흔한 사용법은 아니지만, 중복된 code 패턴을 피하는데 도움이 될 수 있습니다.

In [62]:
#원래 코드가 이렇다면
def a(x, debug=False):  
    if debug:    
        print('Calling a') 
        '...'
        
def b(x, y, z, debug=False):
    if debug:   
        print('Calling b')   
        '...'

def c(x, y, debug=False): 
    if debug:    
        print('Calling c') 
        '...'
        
# 이렇게 refactoring 할수도
@optional_debug
def a(x):    
    '...'
@optional_debug 
def b(x, y, z):    
    '...'

@optional_debug
def c(x, y):    
    '...' 

- 위처럼 keyword-only arguemnt를 주입하는 방식은 만약에 wrapped function에 같은 이름의 keyword가 있다면 충돌날 수 있음. 그래서 다음과 같이 예외처리를 해줘야 할수도 있습니다.

In [63]:
def optional_debug(func):
    if 'debug' in inspect.getargspec(func).args:      
        raise TypeError('debug argument already defined')
    
    @wraps(func) 
    def wrapper(*args, debug=False, **kwargs): 
        if debug:   
            print('Calling', func.__name__)  
        return func(*args, **kwargs) 
    return wrapper

- function signature까지 맞춰주면 금상 첨화!

In [64]:
import inspect

@optional_debug
def add(x, y):
    return x + y

print(inspect.signature(add))

(x, y)


C:\Users\dusol\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  


In [69]:
from functools import wraps 
import inspect

def optional_debug(func):  
    if 'debug' in inspect.getargspec(func).args:  
        raise TypeError('debug argument already defined')
        
    @wraps(func) 
    def wrapper(*args, debug=False, **kwargs): 
        if debug:   
            print('Calling', func.__name__)   
        return func(*args, **kwargs)
        
    sig = inspect.signature(func)  
    parms = list(sig.parameters.values())
    parms.append(inspect.Parameter('debug',  
                                   inspect.Parameter.KEYWORD_ONLY,
                                   default=False))    
    wrapper.__signature__ = sig.replace(parameters=parms) 
    return wrapper 

In [70]:
@optional_debug
def add(x, y):
    return x + y

C:\Users\dusol\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  """


In [71]:
print(inspect.signature(add))

(x, y, *, debug=False)
